# Recursos
Bathia (2010) <br>
Horta (2018) <br>
Kokoszka (2019)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# data
lqdensities_path = "../data/processed/lqdensities.xlsx"

lqdensities = pd.read_excel(lqdensities_path, index_col="x")
lqdensities.head()

,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,...,t_112,t_113,t_114,t_115,t_116,t_117,t_118,t_119,t_120,t_121
x,,,,,,,,,,,,,,,,,,,,,
0.000000,-3.510057,-4.112028,-3.245852,-3.451768,-3.010697,-2.416859,-2.941917,-3.757795,-4.049503,-2.462951,...,-4.106067,-3.044384,-3.499390,-3.806129,-3.256140,-3.627464,-3.130958,-2.843934,-2.740841,-2.971827
0.005025,-3.831881,-4.280989,-3.595798,-3.755390,-3.047621,-1.405972,-2.863023,-4.039823,-4.279095,-2.480686,...,-4.258328,-3.449396,-3.708159,-3.974514,-3.448629,-3.867496,-3.486383,-2.981364,-2.808421,-3.219460
0.010050,-4.056523,-4.415420,-3.900718,-3.986254,-3.194371,-2.904361,-3.497128,-4.238690,-4.443333,-2.980616,...,-4.371172,-3.737058,-3.837260,-4.091067,-3.596603,-4.040252,-3.728953,-2.983511,-2.846394,-3.431278
0.015075,-4.234897,-4.528753,-4.172718,-4.185588,-3.444439,-3.536240,-4.004996,-4.391251,-4.567315,-3.386732,...,-4.458680,-3.952467,-3.922403,-4.180115,-3.821583,-4.184463,-3.907602,-3.041303,-2.974170,-3.652233
0.020101,-4.385658,-4.626079,-4.403389,-4.360551,-3.745242,-3.871029,-4.325151,-4.514555,-4.664294,-3.631419,...,-4.528478,-4.119930,-3.981792,-4.254348,-4.085199,-4.312487,-4.044709,-3.367491,-3.397260,-3.870436


In [3]:
# Defining the inner product on L^2([a,b])
# inner.product = function(f, g, du) {drop(crossprod(f, g))*du}
def inner_product(f, g, du):
    """
    f, g = vectors
    """
    ip = (f.T @ g)*du
    return ip

#The du in this context represents the differential element or step size in numerical integration, 
# specifically for approximating the inner product in the function space L²([a,b]).
# du is the spacing between sample points

In [4]:
# Defining the L2-norm
# L2norm = function(f, du) {sqrt(inner.product(f, f, du))}
def L2norm(f,du):
    norm = np.sqrt(inner_product(f, f, du))
    return norm

In [5]:
# Nxp: row=number_of_daily_observations and p=number_of_curves

#3 curves with dimension 2
matrixX = np.array([[1, 2],
                    [3, 4],
                    [5, 6]])

# 4 curves with dimension 5
Y = np.array([  [2, 1, 3, 4],
                [1, 0, 2, 1],
                [3, 2, 1, 0],
                [1, 2, 3, 4],
                [4, 3, 2, 1]])

In [107]:
 # Interval length (based on equal distance between grid points)
# m = 5001
# du = u[2] - u[1]
du = lqdensities.reset_index()["x"].diff()[1]
du = 0.005
p=5 # script original
p=2 # para poder utilizar matriz pequena
#m = 5001 # number of grid points
m = Y.shape[0]
lag_max = 3

# Y : matrix
#  n = N = ncol(Y)
n = N = Y.shape[1]

# no_boot = B = 1000 # numero de bootstraps
no_boot = B = 10 # numero de bootstraps

alpha = 0.10

dimension = D_val = 2

u = gridpoints = np.linspace(0, 1, m)

In [8]:
# Defining Ybar
# Ybar = rowMeans(Y)
Ybar = np.mean(Y, axis=1, keepdims=True) # rowMeans (point-wise mean)
Ybar

array([[2.5],
       [1. ],
       [1.5],
       [2.5],
       [2.5]])

In [9]:
# Defining the deviation function Ydev = Y - Ybar, which is used as an input
# by the function inner.product in constructing the matrix Kstar
# Ydev = Y - Ybar

Ydev = Y - Ybar
Ydev

array([[-0.5, -1.5,  0.5,  1.5],
       [ 0. , -1. ,  1. ,  0. ],
       [ 1.5,  0.5, -0.5, -1.5],
       [-1.5, -0.5,  0.5,  1.5],
       [ 1.5,  0.5, -0.5, -1.5]])

In [10]:
    # 3. Creating the matrix 'Kstar'    
    # Building the 'core' matrices of Kstar. Below we define the matrices core (n x n),
    # Kstar.core0 [(n-p) x (n-p)] and the array Kstar.core [(n-p) x (n-p) x p].
    # We have that
    #   Kstar = (Kstar.core[,,1] + ... + Kstar.core[,,p])%*%Kstar.core0
    # Where
    #   Kstar.core0[t,s] = <Y[,t],Y[,s]>, t,s in {1,...,n-p}
    #   Kstar.core[t,s,k] = <Y[,t+k],Y[,s+k]>, t,s in {1,...,n-p} and k in {1,...,p}
    # Thus, the matrix core, defined by
    #   core[t,s] = <Y[,t],Y[,s]>, t,s in {1,...,n}
    # contains all the relevant information regarding Kstar.core0 and Kstar.core,
    # and may be used as a building block for the latter matrices.

    # core = inner.product(Ydev,Ydev, du)
    # Kstar.core0 = core[1:(n-p),1:(n-p)]
    # Kstar.core = array(0,c(n-p,n-p,p))
    # for (k in 1:p) Kstar.core[,,k] = core[(k+1):(n-(p-k)),(k+1):(n-(p-k))]

core = inner_product(Ydev, Ydev, du=0.05)
core

array([[ 0.35 ,  0.15 , -0.125, -0.375],
       [ 0.15 ,  0.2  , -0.125, -0.225],
       [-0.125, -0.125,  0.1  ,  0.15 ],
       [-0.375, -0.225,  0.15 ,  0.45 ]])

In [11]:
Kstar_core0 = core[:(n-p),:(n-p)]
Kstar_core0

array([[0.35, 0.15],
       [0.15, 0.2 ]])

In [12]:
# The first dimension (rows) has size n-p.
# The second dimension (columns) has size n-p.
# The third dimension (slices) has size p

Kstar_core = np.zeros((n-p, n-p, p))
Kstar_core

array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]]])

In [13]:
for k in range(p):
    Kstar_core[:, :, k] = core[k : n-(p-k)-1, k : n-(p-k)-1]

In [14]:
Kstar_core

array([[[0.35, 0.2 ],
        [0.35, 0.2 ]],

       [[0.35, 0.2 ],
        [0.35, 0.2 ]]])

In [15]:
Kstar_core = np.zeros((n-p, n-p, p))
Kstar_core

array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]]])

In [16]:
for k in range(p):
    row_index = k+1
    col_index = n-(p-k)+1
    Kstar_core[:, :, k] = core[row_index:col_index , row_index:col_index]

In [17]:
Kstar_core

array([[[ 0.2  ,  0.1  ],
        [-0.125,  0.15 ]],

       [[-0.125,  0.15 ],
        [ 0.1  ,  0.45 ]]])

In [18]:
#print(Kstar_core) não corresponde ao print do R. É necessário printar um de cada vez, já que o numpy faz o display
#de arrays 3D.
for k in range(p):
    print(f"\nSlice {k+1}:")
    print(Kstar_core[:, :, k])


Slice 1:
[[ 0.2   -0.125]
 [-0.125  0.1  ]]

Slice 2:
[[0.1  0.15]
 [0.15 0.45]]


In [19]:
# Summing the matrices in 'Kstar.core'
# Kstar.sum = matrix(0,nrow=n-p,ncol=n-p)
# for (k in 1:p) Kstar.sum = Kstar.sum + Kstar.core[,,k]

Kstar_sum = np.zeros([(n-p),(n-p)]) 
for k in range(p):
    Kstar_sum = Kstar_sum + Kstar_core[:,:,k]
Kstar_sum

array([[0.3  , 0.025],
       [0.025, 0.55 ]])

In [20]:
# Defining Kstar
# Kstar = (n-p)^(-2) * Kstar.sum %*% Kstar.core0
Kstar = ((n-p)**(-2)) * (Kstar_sum.T @ Kstar_core0)
Kstar

array([[0.0271875, 0.0125   ],
       [0.0228125, 0.0284375]])

In [21]:
# 4. Eigen-analisys

# Getting the eigenvalues and eigenvectors from 'Kstar'

# Storing the eigenvalues; length(thetahat)=n-p
# thetahat = eigen(Kstar)$values
# Storing the eigenvectors; each column of gammahat corresponds to
# one eigenvector; dim(gammahat)=[(n-p) x (n-p)]
# gammahat = eigen(Kstar)$vectors
eigenvalues, eigenvectors = np.linalg.eig(Kstar)
idx = eigenvalues.argsort()[::-1] 
thetahat = eigenvalues[idx]
gammahat = eigenvectors[:,idx]

In [22]:
# Defining a tolerance level for 'testing' if the imaginary part of the
# eigenvalues and eigenvectors is zero
tol = 10**(-4)

# Checking if there are any complex eigenvalues (among the first eleven)
# for (j in 1:10){
#     if (abs(Im(thetahat[j]))>tol) print("Complex eigenvalue found.")
# }

for j in range(np.min([len(thetahat),10])):
    if (abs(thetahat[j].imag)>tol):
        print("Complex eigenvalue found.")

thetahat = thetahat.real
gammahat = gammahat.real # diferença para código original, apra não ter que aplicar Re de novo na parte de bootstrap

# Parte abaixo serve apenas para ordenar autovalores e autovetores conjuntamente do maior para o menor, o que já foi feito
# thetahat = sort(thetahat, index.return=TRUE, decreasing=TRUE)
# thetahat.index = thetahat$ix
# thetahat = thetahat$x
# gammahat.temp = matrix(0, nrow=nrow(gammahat), ncol=ncol(gammahat))
# for(j in 1:(length(thetahat)))
# {
#   gammahat.temp[,j] = gammahat[,thetahat.index[j]]
# }
# gammahat = gammahat.temp

In [23]:
# # Storing the original eigenvalues and eigenvectors
# thetahat.old = thetahat
# gammahat.old = gammahat
thetahat_old = thetahat
gammahat_old = gammahat

In [28]:
def sampleCols(A):
    """
    Randomly shuffle columns of a matrix.
    
    Parameters:
    A : numpy.ndarray
        Input matrix (2D array)
    
    Returns:
    numpy.ndarray : Matrix with columns randomly shuffled
    """
    # Get number of columns
    n_cols = A.shape[1]
    
    # Generate random permutation of column indices
    idx = np.random.permutation(n_cols)
    
    # Create new matrix with shuffled columns
    M = A[:, idx].copy()
    
    return M

In [49]:
# Selecting the number of components
# if(select_ncomp == "TRUE")
select_ncomp = True
lag_max = 2
if select_ncomp == True:
    bs_pvalues = np.full(lag_max, 0)
    # Building the estimated functions Yhat with dimension d0, d0 in {1,2,...,lag_max}.
    # See the main code for an explanation of this section.
    # for(d0 in 1:lag_max)
    # {
    # thetahatH0 = Re(thetahat.old[d0+1])
    # thetahat = Re(thetahat.old[1:d0])
    # gammahat = Re(gammahat.old[,1:d0])
    # psihat.root = Ydev[,1:(n-p)] %*% gammahat
    # psihat = matrix(0, nrow = m, ncol = d0)
    # for(i in 1:d0) psihat[,i] = psihat.root[,i]/L2norm(psihat.root[,i], du = du); rm(i)
    for d0 in range(lag_max-1): # adicionado -1 para slicing não ultrapassar limites. Ver teste de hipótese do artigo (lambda_{d0+1}=0)
        # print(d0)
        thetahatH0 = thetahat_old[d0+1]
        thetahat   = thetahat_old[d0]
        gammahat   = gammahat_old[:,:(d0+1)]
        psihat_root = Ydev[:,:(n-p)] @ gammahat_old[:,:(d0+1)]
        psihat = np.zeros([m, d0+1])
        # for(i in 1:d0) psihat[,i] = psihat.root[,i]/L2norm(psihat.root[,i], du = du); rm(i)
        for i in range(d0+1):
            print(i)
            psihat[ : , i ] = psihat_root[:,i]/L2norm(psihat_root[:,i], du=du)
            # del i
        # etahat = inner.product(psihat, Ydev, du = du)
        etahat = inner_product(psihat, Ydev, du=du)
        # Yhat = Ybar + psihat %*% etahat
        Yhat = Ybar + (psihat @ etahat) 

        # Correção de densidades negativas. Pela minha abordagem não deveria existir; jogar print se tiver.
        # Yhat.fix = Yhat
        # Yhat.fix[Yhat.fix < 0] = 0
        Yhat_fix = Yhat
        if (Yhat_fix[Yhat_fix<0]).size != 0:
            print(Yhat_fix)
            print("Negative values were found; substituting them with 0.")
            Yhat_fix[np.where(Yhat_fix < 0)] = 0
            # for (t in 1:N) Yhat.fix[,t] = Yhat.fix[,t]/(sum(Yhat.fix[,t])*du); rm(t)
            for t in np.range(N):
                Yhat_fix[:,t] = Yhat_fix[:,t]/(sum(Yhat_fix[:,t])*du) # normaliza densidade depois de substituir por zero (soma=1 ~ pdf)
        
        epsilonhat = Y - Yhat_fix
    
        # Let bootstrap begin   
        # sampleCols = function(A)
        # {
        #   idx = sample(1:ncol(A))
        #   M = matrix(0, nrow = nrow(A), ncol = ncol(A))
        #   for(i in 1:length(idx))
        #   {
        #     M[,i] = A[,idx[i]]
        #   }
        #   return(M)
        # }

        # bs_thetahat = NP.vector("numeric", B)
        bs_thetahat = np.full(B,0)
        bs = {}
        for i in range(B):
            # This section of the code repeats c-main.r, however inside the list object bs
            # Building the bootstrap 'observed' function Y
            # bs$epsilon = sampleCols(epsilonhat)
            # bs$Y = Yhat.fix + bs$epsilon
            # bs$Ybar = rowMeans(bs$Y)
            # bs$Ydev = bs$Y - bs$Ybar
            # bs$core = inner.product(bs$Ydev, bs$Ydev, du = du)
            # bs$Kstar.core0 = bs$core[1:(n-p),1:(n-p)]
            # bs$Kstar.core = array(0,c(n-p,n-p,p))
            # for(k in 1:p) bs$Kstar.core[,,k] = bs$core[(k+1):(n-(p-k)),(k+1):(n-(p-k))]
            # bs$Kstar.sum = matrix(0, nrow=n-p, ncol=n-p)
            # for(k in 1:p) bs$Kstar.sum = bs$Kstar.sum + bs$Kstar.core[,,k]
            # bs$Kstar = (n-p)^(-2) * bs$Kstar.sum %*% bs$Kstar.core0
            # bs$thetahat = eigen(bs$Kstar)$values
            # bs$thetahat = sort(bs$thetahat, decreasing = TRUE)
            bs["epsilon"] = sampleCols(epsilonhat)
            bs["Y"]       = Yhat_fix + bs["epsilon"]
            bs["Ybar"]    = np.mean(bs["Y"], axis=1, keepdims=True)
            bs["Ydev"]    = bs["Y"] - bs["Ybar"]
            bs["core"]    = inner_product(bs["Ydev"], bs["Ydev"], du = du)
            bs["Kstar_core0"] = bs["core"][:(n-p),:(n-p)]
            bs["Kstar_core"]  = np.zeros((n-p, n-p, p))
            for k in range(p):
                bs["Kstar_core"][:,:,k] = bs["core"][(k+1):(n-(p-k)),(k+1):(n-(p-k))]
            bs["Kstar_sum"] = np.zeros([n-p,n-p])
            for k in range(p):
                bs["Kstar_sum"] = bs["Kstar_sum"] + bs["Kstar_core"][:,:,k]
            bs["Kstar"] = (n-p)**(-2) * bs["Kstar_sum"] @ bs["Kstar_core0"]
            eigenvalues, _ = np.linalg.eig(bs["Kstar"])
            idx = eigenvalues.argsort()[::-1] 
            bs["thetahat"] = eigenvalues[idx]
            # Storing the (d0+1)-th eigenvalue obtained in the i-th bootstrap loop
            bs_thetahat[i] = bs["thetahat"][d0+1]
        # Storing the p-values
        # bs.pvalues[d0] = sum(bs.thetahat >= thetahatH0)/B
        bs_pvalues[d0] = (bs_thetahat >= thetahatH0).sum()/B
    # d0 = head(which(bs.pvalues < alpha))[1] + 1
    d0 = np.where(bs_pvalues < alpha)[0][0] + 1
    if np.isnan(d0):
        d0 = 1
if select_ncomp == False:
    d0 = dimension

0


In [77]:
# 5. Defining the estimator Yhat
# Storing only the d0 largest eigenvalues and the associated eigenvectors
# thetahat = Re(thetahat.old[1:d0]) # length(thetahat) = d0
# gammahat = Re(gammahat.old[,1:d0]) # dim(gammahat) = [(n-p) x d0]
thetahat = (thetahat_old[:d0+1]).real # length(thetahat) = d0
gammahat = (gammahat_old[:,:d0+1]).real # dim(gammahat) = [(n-p) x d0]

In [ ]:
# Defining the eigenfunctions psihat.root. These are the functions
# given in equation (2.13)
#   psihat.root[u,j] = gammahat[1,j]*Ydev[u,1] + ... + gammahat[n-p,j]*Ydev[u,n-p]
# Note that dim(psihat.root) = [m x d0]. These functions are not
# necessarily orthonormal.
# psihat.root = Ydev[,1:(n-p)]%*%gammahat
psihat_root = Ydev[:,:(n-p)] @ gammahat

In [85]:
# Normalizing (tested for orthogonality; already orthogonal to each other)
# psihat = matrix(0,nrow=m,ncol=d0)
psihat = np.zeros([m,d0+1])

In [100]:
# for (i in 1:d0) psihat[,i] = psihat.root[,i]/L2norm(psihat.root[,i], du)
for i in range((d0+1)):
    print(i)
    psihat[:,i] = psihat_root[:,i]/L2norm(psihat_root[:,i],du)

0


In [102]:
# Defining etahat (d0 x n). We have that
#   etahat[j,t]=<Ydev[,t],psihat[,j]>.
# This is a d0-vector process. Time varies columnwise.
# etahat = inner.product(psihat,Ydev, du)
etahat = inner_product(psihat, Ydev, du)

In [ ]:
# Defining Yhat (m x n). The (u,t)-th element of Yhat is given by
#   Yhat[u,t] = Ybar[u] + psihat[u,1]*etahat[1,t] + ... + psihat[u,d0]*etahat[d0,t]
# Yhat = Ybar + psihat%*%etahat
Yhat = Ybar + psihat @ etahat

In [104]:
# # Note that no restrictions were made upon Yhat, so it may happen that
# sum(Yhat[,t])*du!=1 and/or Yhat[u,t]<0.
# We define Yhat.fix to meet these restrictions.
# Yhat.fix = Yhat
# Yhat.fix[Yhat.fix < 0] = 0
# for (t in 1:N) Yhat.fix[,t] = Yhat.fix[,t]/(sum(Yhat.fix[,t])*du); rm(t)

# Correção de densidades negativas. Pela minha abordagem não deveria existir; jogar print se tiver.
Yhat_fix = Yhat
if (Yhat_fix[Yhat_fix<0]).size != 0:
    print(Yhat_fix)
    print("Negative values were found; substituting them with 0.")
    Yhat_fix[np.where(Yhat_fix < 0)] = 0
    # for (t in 1:N) Yhat.fix[,t] = Yhat.fix[,t]/(sum(Yhat.fix[,t])*du); rm(t)
    for t in np.range(N):
        Yhat_fix[:,t] = Yhat_fix[:,t]/(sum(Yhat_fix[:,t])*du)

In [105]:
# Defining epsilonhat (m x n)
epsilonhat = Y - Yhat

In [ ]:
if (select_ncomp == True):
    result = {
        'Y': Y,
        'Ybar': Ybar, 
        'thetahat': thetahat,
        'gammahat': gammahat,
        'psihat': psihat,
        'etahat': etahat,
        'Yhat': Yhat,
        'Yhat_fix': Yhat_fix,
        'epsilonhat': epsilonhat,
        'u': u,
        'd0': d0,
        'bs_pvalues': bs_pvalues
    }
elif (select_ncomp == False):
    result = {
        'Y': Y,
        'Ybar': Ybar, 
        'thetahat': thetahat,
        'gammahat': gammahat,
        'psihat': psihat,
        'etahat': etahat,
        'Yhat': Yhat,
        'Yhat_fix': Yhat_fix, 
        'epsilonhat': epsilonhat,
        'u': u,
        'd0': d0
    }

In [ ]:
def inner_product(f, g, du):
    """
    f, g = vectors
    """
    ip = (f.T @ g)*du
    return ip

def L2norm(f,du):
    norm = np.sqrt(inner_product(f, f, du))
    return norm

def sampleCols(A):
    """
    Randomly shuffle columns of a matrix.
    
    Parameters:
    A : numpy.ndarray
        Input matrix (2D array)
    
    Returns:
    numpy.ndarray : Matrix with columns randomly shuffled
    """
    # Get number of columns
    n_cols = A.shape[1]
    
    # Generate random permutation of column indices
    idx = np.random.permutation(n_cols)
    
    # Create new matrix with shuffled columns
    M = A[:, idx].copy()
    
    return M

def super_fun(
        Y,
        lag_max,
        B,
        alpha,
        du,
        p,
        m,
        u,
        select_ncomp,
        dimension
        ) -> dict:
    Ybar = np.mean(Y, axis=1, keepdims=True)
    Ydev = Y - Ybar
    core = inner_product(Ydev, Ydev, du=0.05)
    Kstar_core0 = core[:(n-p),:(n-p)]
    Kstar_core = np.zeros((n-p, n-p, p))
    for k in range(p):
        Kstar_core[:, :, k] = core[k : n-(p-k)-1, k : n-(p-k)-1]
    Kstar_sum = np.zeros([(n-p),(n-p)]) 
    for k in range(p):
        Kstar_sum = Kstar_sum + Kstar_core[:,:,k]
    Kstar = ((n-p)**(-2)) * (Kstar_sum.T @ Kstar_core0)
    eigenvalues, eigenvectors = np.linalg.eig(Kstar)
    idx = eigenvalues.argsort()[::-1] 
    thetahat = eigenvalues[idx]
    gammahat = eigenvectors[:,idx]
    tol = 10**(-4)
    for j in range(np.min([len(thetahat),10])):
        if (abs(thetahat[j].imag)>tol):
            print("Complex eigenvalue found.")
    thetahat = thetahat.real
    gammahat = gammahat.real
    thetahat_old = thetahat
    gammahat_old = gammahat

    if select_ncomp:
        bs_pvalues = np.full(lag_max, 0)
        for d0 in range(lag_max-1): 
            thetahatH0 = thetahat_old[d0+1]
            thetahat   = thetahat_old[d0]
            gammahat   = gammahat_old[:,:(d0+1)]
            psihat_root = Ydev[:,:(n-p)] @ gammahat_old[:,:(d0+1)]
            psihat = np.zeros([m, d0+1])
            for i in range(d0+1):
                print(i)
                psihat[ : , i ] = psihat_root[:,i]/L2norm(psihat_root[:,i], du=du)
            etahat = inner_product(psihat, Ydev, du=du)
            Yhat = Ybar + (psihat @ etahat) 
            Yhat_fix = Yhat
            if (Yhat_fix[Yhat_fix<0]).size != 0:
                print(Yhat_fix)
                print("Negative values were found; substituting them with 0.")
                Yhat_fix[np.where(Yhat_fix < 0)] = 0
                for t in np.range(N):
                    Yhat_fix[:,t] = Yhat_fix[:,t]/(sum(Yhat_fix[:,t])*du) 
            epsilonhat = Y - Yhat_fix

            bs_thetahat = np.full(B,0)
            bs = {}
            for i in range(B):
                bs["epsilon"] = sampleCols(epsilonhat)
                bs["Y"]       = Yhat_fix + bs["epsilon"]
                bs["Ybar"]    = np.mean(bs["Y"], axis=1, keepdims=True)
                bs["Ydev"]    = bs["Y"] - bs["Ybar"]
                bs["core"]    = inner_product(bs["Ydev"], bs["Ydev"], du = du)
                bs["Kstar_core0"] = bs["core"][:(n-p),:(n-p)]
                bs["Kstar_core"]  = np.zeros((n-p, n-p, p))
                for k in range(p):
                    bs["Kstar_core"][:,:,k] = bs["core"][(k+1):(n-(p-k)),(k+1):(n-(p-k))]
                bs["Kstar_sum"] = np.zeros([n-p,n-p])
                for k in range(p):
                    bs["Kstar_sum"] = bs["Kstar_sum"] + bs["Kstar_core"][:,:,k]
                bs["Kstar"] = (n-p)**(-2) * bs["Kstar_sum"] @ bs["Kstar_core0"]
                eigenvalues, _ = np.linalg.eig(bs["Kstar"])
                idx = eigenvalues.argsort()[::-1] 
                bs["thetahat"] = eigenvalues[idx]
                bs_thetahat[i] = bs["thetahat"][d0+1]
            bs_pvalues[d0] = (bs_thetahat >= thetahatH0).sum()/B
        d0 = np.where(bs_pvalues < alpha)[0][0] + 1
        if np.isnan(d0):
            d0 = 1
    else:
        d0 = dimension
        
    thetahat = (thetahat_old[:d0+1]).real
    gammahat = (gammahat_old[:,:d0+1]).real
    psihat_root = Ydev[:,:(n-p)] @ gammahat
    psihat = np.zeros([m,d0+1])
    for i in range((d0+1)):
        psihat[:,i] = psihat_root[:,i]/L2norm(psihat_root[:,i],du)
    etahat = inner_product(psihat, Ydev, du)
    Yhat = Ybar + psihat @ etahat
    Yhat_fix = Yhat
    if (Yhat_fix[Yhat_fix<0]).size != 0:
        print(Yhat_fix)
        print("Negative values were found; substituting them with 0.")
        Yhat_fix[np.where(Yhat_fix < 0)] = 0
        for t in np.range(N):
            Yhat_fix[:,t] = Yhat_fix[:,t]/(sum(Yhat_fix[:,t])*du)
    epsilonhat = Y - Yhat_fix

    result = {
        "Y": Y,
        "Ybar": Ybar,
        "thetahat": thetahat,
        "gammahat": gammahat,
        "psihat": psihat,
        "etahat": etahat,
        "Yhat": Yhat,
        "Yhat_fix": Yhat_fix,
        "epsilonhat": epsilonhat,
        "u": u,
        "d0": d0
        }    
    if select_ncomp:
        result["bs_pvalues"] = bs_pvalues

    return result